# Entrenando RL para equilibrar el Cartpole

Este cuaderno forma parte del [Currículo de IA para Principiantes](http://aka.ms/ai-beginners). Se ha inspirado en el [tutorial oficial de PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) y en [esta implementación de Cartpole en PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

En este ejemplo, utilizaremos RL para entrenar un modelo que pueda equilibrar un poste sobre un carrito que puede moverse hacia la izquierda y la derecha en una escala horizontal. Usaremos el entorno de [OpenAI Gym](https://www.gymlibrary.ml/) para simular el poste.

> **Nota**: Puedes ejecutar el código de esta lección localmente (por ejemplo, desde Visual Studio Code), en cuyo caso la simulación se abrirá en una nueva ventana. Si ejecutas el código en línea, es posible que necesites hacer algunos ajustes al código, como se describe [aquí](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Comenzaremos asegurándonos de que Gym esté instalado:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Ahora vamos a crear el entorno CartPole y ver cómo interactuar con él. Un entorno tiene las siguientes propiedades:

* **Action space** es el conjunto de acciones posibles que podemos realizar en cada paso de la simulación.
* **Observation space** es el espacio de observaciones que podemos realizar.


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Veamos cómo funciona la simulación. El siguiente bucle ejecuta la simulación hasta que `env.step` no devuelva el indicador de terminación `done`. Elegiremos acciones de forma aleatoria utilizando `env.action_space.sample()`, lo que significa que el experimento probablemente fallará muy rápido (el entorno CartPole termina cuando la velocidad del CartPole, su posición o su ángulo están fuera de ciertos límites).

> La simulación se abrirá en una nueva ventana. Puedes ejecutar el código varias veces y observar cómo se comporta.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Puedes notar que las observaciones contienen 4 números. Estos son:
- Posición del carrito
- Velocidad del carrito
- Ángulo del poste
- Tasa de rotación del poste

`rew` es la recompensa que recibimos en cada paso. Puedes ver que en el entorno de CartPole se otorga 1 punto por cada paso de simulación, y el objetivo es maximizar la recompensa total, es decir, el tiempo que CartPole puede mantenerse equilibrado sin caerse.

Durante el aprendizaje por refuerzo, nuestro objetivo es entrenar una **política** $\pi$, que para cada estado $s$ nos indicará qué acción $a$ tomar, esencialmente $a = \pi(s)$.

Si deseas una solución probabilística, puedes pensar en la política como un conjunto de probabilidades para cada acción, es decir, $\pi(a|s)$ representaría la probabilidad de que debamos tomar la acción $a$ en el estado $s$.

## Método de Gradiente de Política

En el algoritmo más simple de aprendizaje por refuerzo, llamado **Gradiente de Política**, entrenaremos una red neuronal para predecir la próxima acción.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Entrenaremos la red realizando muchos experimentos y actualizando nuestra red después de cada ejecución. Definamos una función que ejecutará el experimento y devolverá los resultados (el llamado **rastro**) - todos los estados, acciones (y sus probabilidades recomendadas) y recompensas:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Puedes ejecutar un episodio con una red no entrenada y observar que la recompensa total (también conocida como la duración del episodio) es muy baja:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Uno de los aspectos complicados del algoritmo de gradiente de política es usar **recompensas descontadas**. La idea es que calculamos el vector de recompensas totales en cada paso del juego, y durante este proceso descontamos las recompensas iniciales utilizando algún coeficiente $gamma$. También normalizamos el vector resultante, porque lo usaremos como peso para afectar nuestro entrenamiento:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

¡Ahora vamos a entrenar! Ejecutaremos 300 episodios, y en cada episodio haremos lo siguiente:

1. Ejecutar el experimento y recopilar el rastro.
1. Calcular la diferencia (`gradients`) entre las acciones realizadas y las probabilidades predichas. Cuanto menor sea la diferencia, más seguros estaremos de haber tomado la acción correcta.
1. Calcular las recompensas descontadas y multiplicar los gradientes por las recompensas descontadas. Esto asegurará que los pasos con mayores recompensas tengan un mayor impacto en el resultado final que aquellos con recompensas más bajas.
1. Las acciones objetivo esperadas para nuestra red neuronal se tomarán en parte de las probabilidades predichas durante la ejecución y en parte de los gradientes calculados. Usaremos el parámetro `alpha` para determinar en qué medida se tienen en cuenta los gradientes y las recompensas; esto se conoce como *tasa de aprendizaje* del algoritmo de refuerzo.
1. Finalmente, entrenamos nuestra red con los estados y las acciones esperadas, y repetimos el proceso.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Ahora ejecutemos el episodio con renderizado para ver el resultado:


In [ ]:
_ = run_episode(render=True)

¡Esperemos que puedas ver que el poste ahora puede equilibrarse bastante bien!

## Modelo Actor-Crítico

El modelo Actor-Crítico es un desarrollo adicional de los gradientes de política, en el cual construimos una red neuronal para aprender tanto la política como las recompensas estimadas. La red tendrá dos salidas (o puedes verlo como dos redes separadas):
* **Actor** recomendará la acción a tomar proporcionándonos la distribución de probabilidad del estado, como en el modelo de gradiente de política.
* **Crítico** estimará cuál sería la recompensa de esas acciones. Devuelve las recompensas totales estimadas en el futuro en el estado dado.

Definamos un modelo de este tipo:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Necesitaríamos modificar ligeramente nuestras funciones `discounted_rewards` y `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Ahora ejecutaremos el bucle principal de entrenamiento. Utilizaremos un proceso de entrenamiento manual de la red calculando las funciones de pérdida adecuadas y actualizando los parámetros de la red:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Conclusión

Hemos visto dos algoritmos de aprendizaje por refuerzo en esta demostración: el gradiente de política simple y el actor-crítico más sofisticado. Puedes observar que estos algoritmos operan con nociones abstractas de estado, acción y recompensa, lo que les permite aplicarse a entornos muy diferentes.

El aprendizaje por refuerzo nos permite aprender la mejor estrategia para resolver un problema simplemente observando la recompensa final. El hecho de que no necesitemos conjuntos de datos etiquetados nos permite repetir simulaciones muchas veces para optimizar nuestros modelos. Sin embargo, todavía existen muchos desafíos en el aprendizaje por refuerzo, los cuales podrías explorar si decides profundizar en esta interesante área de la inteligencia artificial.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
